In [23]:
# Here we're putting our input parametrs
import cv2
import numpy as np
import pandas as pd

In [24]:
import tensorflow.compat.v1 as tf
FLAGS = tf.app.flags.FLAGS

In [26]:
shuffle = True
localization = FLAGS.is_localization
imageNet_mean_pixel = [103.939, 116.799, 123.68]
global_std = 68.76

IMG_ROWS = 64
IMG_COLS = 64

'shuffle = True\nlocalization = FLAGS.is_localization\nimageNet_mean_pixel = [103.939, 116.799, 123.68]\nglobal_std = 68.76\n\nIMG_ROWS = 64\nIMG_COLS = 64'

In [27]:
def get_image(path, x1, y1, x2, y2):
    '''
    :param path: image path
    :param x1: the upper left and lower right coordinates to localize the apparels
    :param y1:
    :param x2:
    :param y2:
    :return: a numpy array with dimensions [img_row, img_col, img_depth]
    '''
    img = cv2.imread(path)
    if localization is True:
        if img is None or img.shape[0] == 0 or img.shape[1] == 0:
            img = np.zeros((1, IMG_ROWS, IMG_COLS, 0))
        img = cv2.resize(img, (IMG_ROWS, IMG_COLS))
        assert img.shape == (IMG_ROWS, IMG_COLS, 3)
    else:
        img = cv2.resize(img, (IMG_ROWS, IMG_COLS))

    img = img.reshape(1, IMG_ROWS, IMG_COLS, 3)

    return img

"def get_image(path, x1, y1, x2, y2):\n    '''\n    :param path: image path\n    :param x1: the upper left and lower right coordinates to localize the apparels\n    :param y1:\n    :param x2:\n    :param y2:\n    :return: a numpy array with dimensions [img_row, img_col, img_depth]\n    '''\n    img = cv2.imread(path)\n    if localization is True:\n        if img is None or img.shape[0] == 0 or img.shape[1] == 0:\n            img = np.zeros((1, IMG_ROWS, IMG_COLS, 0))\n        img = cv2.resize(img, (IMG_ROWS, IMG_COLS))\n        assert img.shape == (IMG_ROWS, IMG_COLS, 3)\n    else:\n        img = cv2.resize(img, (IMG_ROWS, IMG_COLS))\n\n    img = img.reshape(1, IMG_ROWS, IMG_COLS, 3)\n\n    return img"

In [28]:
def load_data_numpy(df):
    num_images = len(df)
    image_path_array = df['image_path'].as_matrix()
    label_array = df['category'].as_matrix()
    x1 = df['x1_modified'].as_matrix().reshape(-1, 1)
    y1 = df['y1_modified'].as_matrix().reshape(-1, 1)
    x2 = df['x2_modified'].as_matrix().reshape(-1, 1)
    y2 = df['y2_modified'].as_matrix().reshape(-1, 1)
    bbox_array = np.concatenate((x1, y1, x2, y2), axis=1)

    image_array = np.array([]).reshape(-1, IMG_ROWS, IMG_COLS, 3)
    adjusted_std = 1.0/np.sqrt(IMG_COLS * IMG_ROWS * 3)

    for i in range(num_images):
        img = get_image(image_path_array[i], x1=x1[i, 0], y1=y1[i, 0], x2=x2[i, 0], y2=y2[i, 0])
        flip_indicator = np.random.randint(low=0, high=2)
        if flip_indicator == 0:
            img[0, ...] = cv2.flip(img[0, ...], 1)

        image_array = np.concatenate((image_array, img))

    image_array = (image_array - imageNet_mean_pixel) / global_std
 
    assert image_array.shape[1:] == (IMG_ROWS, IMG_COLS, 3)
  
    return image_array, label_array, bbox_array

"def load_data_numpy(df):\n    num_images = len(df)\n    image_path_array = df['image_path'].as_matrix()\n    label_array = df['category'].as_matrix()\n    x1 = df['x1_modified'].as_matrix().reshape(-1, 1)\n    y1 = df['y1_modified'].as_matrix().reshape(-1, 1)\n    x2 = df['x2_modified'].as_matrix().reshape(-1, 1)\n    y2 = df['y2_modified'].as_matrix().reshape(-1, 1)\n    bbox_array = np.concatenate((x1, y1, x2, y2), axis=1)\n\n    image_array = np.array([]).reshape(-1, IMG_ROWS, IMG_COLS, 3)\n    adjusted_std = 1.0/np.sqrt(IMG_COLS * IMG_ROWS * 3)\n\n    for i in range(num_images):\n        img = get_image(image_path_array[i], x1=x1[i, 0], y1=y1[i, 0], x2=x2[i, 0], y2=y2[i, 0])\n        flip_indicator = np.random.randint(low=0, high=2)\n        if flip_indicator == 0:\n            img[0, ...] = cv2.flip(img[0, ...], 1)\n\n        image_array = np.concatenate((image_array, img))\n\n    image_array = (image_array - imageNet_mean_pixel) / global_std\n \n    assert image_array.shape[1:] 

In [29]:
def prepare_df(path, usecols, shuffle=shuffle):
    df = pd.read_csv(path, usecols=usecols)
    if shuffle is True:
        order = np.random.permutation(len(df))
        df = df.iloc[order, :]
    return df

'def prepare_df(path, usecols, shuffle=shuffle):\n    df = pd.read_csv(path, usecols=usecols)\n    if shuffle is True:\n        order = np.random.permutation(len(df))\n        df = df.iloc[order, :]\n    return df'